In [ ]:
#Roll no:DS5B-2125
#This is a classification problem need to be solve using logistic regression

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 38.4 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=0aadd585668286b4d4248e7569f1046bd796d5f494055601aa4d922d0c68a88d
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession
session=SparkSession.builder.appName("HRComma").master("local[2]").getOrCreate()

In [5]:
data=session.read.csv("/content/HR comma (1).csv",header=True, inferSchema=True) #roll no:-2131

In [6]:
data.columns

['satisfaction_level',
 'last_evaluation',
 'number_project',
 'average_montly_hours',
 'time_spend_company',
 'Work_accident',
 'left',
 'promotion_last_5years',
 'sales',
 'salary']

In [7]:
data.show(3)

+------------------+---------------+--------------+--------------------+------------------+-------------+----+---------------------+-----+------+
|satisfaction_level|last_evaluation|number_project|average_montly_hours|time_spend_company|Work_accident|left|promotion_last_5years|sales|salary|
+------------------+---------------+--------------+--------------------+------------------+-------------+----+---------------------+-----+------+
|              0.38|           0.53|             2|                 157|                 3|            0|   1|                    0|sales|   low|
|               0.8|           0.86|             5|                 262|                 6|            0|   1|                    0|sales|medium|
|              0.11|           0.88|             7|                 272|                 4|            0|   1|                    0|sales|medium|
+------------------+---------------+--------------+--------------------+------------------+-------------+----+--------------

In [8]:
from pyspark.ml.feature import VectorAssembler,StringIndexer
str_idx=StringIndexer(inputCols=["sales","salary"],outputCols=["newsales","newsalary"])
#this will convert input col to output col


In [9]:
#Vector assembler is used to merge different cols into 1 col and onlybuses numeric values
#for making different independent variables into 1 independent variable
#henre left being dependent variable can't be used
vec_ass=VectorAssembler(inputCols=["satisfaction_level","number_project","average_montly_hours","time_spend_company","Work_accident","promotion_last_5years","newsales","newsalary"],outputCol="all_features")

In [10]:
#creating a model
#all supervised machine learning model take two arguments: featurescol andlabelcol
from pyspark.ml.classification import LogisticRegression
lr=LogisticRegression(featuresCol="all_features",labelCol="left")   #roll no:-2131

In [11]:
from pyspark.ml import Pipeline
mypipeline=Pipeline(stages=[str_idx,vec_ass,lr])

In [13]:
#Dividing the datasets
training,test=data.randomSplit([0.71,0.29])

In [14]:
#Execute the functions
#training the model
lrmodel=mypipeline.fit(training)

In [15]:
#prediction
results=lrmodel.transform(test)
results.show(5)

+------------------+---------------+--------------+--------------------+------------------+-------------+----+---------------------+-----------+------+--------+---------+--------------------+--------------------+--------------------+----------+
|satisfaction_level|last_evaluation|number_project|average_montly_hours|time_spend_company|Work_accident|left|promotion_last_5years|      sales|salary|newsales|newsalary|        all_features|       rawPrediction|         probability|prediction|
+------------------+---------------+--------------+--------------------+------------------+-------------+----+---------------------+-----------+------+--------+---------+--------------------+--------------------+--------------------+----------+
|              0.09|           0.77|             6|                 244|                 4|            0|   1|                    0|product_mng|   low|     4.0|      0.0|[0.09,6.0,244.0,4...|[-0.9166097349758...|[0.28564918749098...|       1.0|
|              0.09|

In [16]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator
eval=BinaryClassificationEvaluator(rawPredictionCol="rawPrediction",labelCol="left")

In [17]:
#The final result or accuracy
eval.evaluate(results)

0.8132792648006996